In [ ]:
import numpy as np
import requests
from PIL import Image
import cv2
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration, AutoModelForCausalLM, AutoTokenizer, pipeline

# OpenWeatherMap API endpoint
weather_url = "https://api.openweathermap.org/data/2.5/weather"

# Your OpenWeatherMap API key
api_key = "a67752c4dd280bdecc63a72f21110b21"

# Load the Visual Language Model (VLM) for image analysis
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vlm_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
vlm_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

# Load the Language Model (LLM) for text generation
llm_model_id = "Qwen/Qwen2.5-7B-Instruct"
llm_language_model = AutoModelForCausalLM.from_pretrained(
    llm_model_id,
    device_map="cpu"  # Load the model on CPU instead of GPU
)

llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_id)
llm_pipeline = pipeline("text-generation", model=llm_language_model, tokenizer=llm_tokenizer)

# Function to fetch climate data for a given city
def get_weather_data(city_name):
    try:
        params = {"q": city_name, "appid": api_key, "units": "metric"}
        response = requests.get(weather_url, params=params, timeout=10)  # Set timeout for the request
        response.raise_for_status()  # Raise exception for non-200 status codes
        weather_data = response.json()
        temperature = weather_data["main"]["temp"]
        humidity = weather_data["main"]["humidity"]
        rain = weather_data.get("rain", {}).get("1h", 0)
        wind_speed = weather_data["wind"]["speed"]
        return {"temperature": temperature, "humidity": humidity, "rain": rain, "wind": wind_speed}
    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")
        return None

# Function to calculate vegetative land percentage from an image
def calculate_vegetative_land(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image at path {image_path} could not be loaded.")
    image = cv2.resize(image, (640, 480))  # Resize for faster processing
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Green range in HSV (tune for better vegetation detection)
    lower_green = np.array([35, 40, 20])
    upper_green = np.array([85, 255, 255])
    green_mask = cv2.inRange(hsv, lower_green, upper_green)

    # Calculate the percentage of vegetative land
    vegetative_pixels = np.sum(green_mask > 0)
    total_pixels = image.shape[0] * image.shape[1]
    vegetative_percentage = (vegetative_pixels / total_pixels) * 100
    return vegetative_percentage

# Function to calculate Land Vegetative Ratio (LVR) from pre and post urbanization data
def calculate_lvr(pre_vegetative_percentage, post_vegetative_percentage):
    return post_vegetative_percentage / pre_vegetative_percentage if pre_vegetative_percentage > 0 else 0

# Function to predict post-urbanization climate data based on LVR
def predict_post_urbanization_climate(pre_climate_data, lvr):
    predicted_climate = {}
    
    # Linear relationships between LVR and climate factors
    predicted_climate['temperature'] = pre_climate_data['temperature'] * (1 + (1 - lvr) * 0.2)  # Temp increases as vegetation decreases
    predicted_climate['humidity'] = pre_climate_data['humidity'] * (lvr)  # Humidity decreases as vegetation decreases
    predicted_climate['rain'] = pre_climate_data['rain'] * (lvr)  # Rain decreases as vegetation decreases
    predicted_climate['wind'] = pre_climate_data['wind'] * (1 + (1 - lvr) * 0.1)  # Wind increases as vegetation decreases

    return predicted_climate

# Function to calculate the difference between pre-urbanization and predicted post-urbanization climate
def calculate_climate_difference(pre_climate_data, post_climate_data):
    delta_temp = post_climate_data['temperature'] - pre_climate_data['temperature']
    delta_humidity = post_climate_data['humidity'] - pre_climate_data['humidity']
    delta_rain = post_climate_data['rain'] - pre_climate_data['rain']
    delta_wind = post_climate_data['wind'] - pre_climate_data['wind']

    return delta_temp, delta_humidity, delta_rain, delta_wind

# Function to calculate Biodiversity Risk Score (BRS)
def calculate_brs(delta_temp, delta_humidity, delta_rain, delta_wind, weights):
    # Apply climate functions to each factor
    temp_score = np.log(1 + abs(delta_temp))
    humidity_score = np.exp(-abs(delta_humidity))
    rain_score = np.exp(-abs(delta_rain))
    wind_score = abs(delta_wind)

    # Weighted sum of climate impacts
    weighted_score = (weights['temp'] * temp_score +
                      weights['humidity'] * humidity_score +
                      weights['rain'] * rain_score +
                      weights['wind'] * wind_score)

    return weighted_score 

# Function to generate suggestions using LLM
def generate_llm_suggestions(metrics, map_diff):
    prompt = f"""
    System: You are an expert in urban planning, environmental preservation, and sustainable development. 
    Your task is to provide actionable recommendations for optimizing urbanization to balance ecological preservation and development needs.

    User: Based on the following data:

    Biodiversity Loss (Risk Score): {metrics['biodiversity_risk_score']} 
    Temperature Increase: {metrics['temperature_increase']}°C
    Humidity Change: {metrics['humidity_change']}%
    Rainfall Change: {metrics['rain_change']} mm
    Wind Speed Change: {metrics['wind_speed_change']} m/s
    Vegetative Land Percentage (Pre-Urbanization): {metrics['pre_vegetative_percentage']}%
    Vegetative Land Percentage (Post-Urbanization): {metrics['post_vegetative_percentage']}%

    Additionally, here is the observed difference in pre-urbanization and post-urbanization maps:
    {map_diff}

    Using these insights, propose a detailed urban development strategy that includes:

    1. Green Space Optimization
    2. Biodiversity Corridors
    3. Development Layout Adjustments
    4. Sustainable Design Solutions

    Suggest how to make the post-urbanization map better in terms of environmental sustainability and biodiversity preservation.
    """

    result = llm_pipeline(prompt, max_new_tokens=256, temperature=0.7, top_p=0.85)
    return result[0]['generated_text']

# Main function
def main():
    # Input data
    pre_urbanization_image_path = "/kaggle/input/hehehe/before_urban.jpg"  # Example path
    post_urbanization_image_path = "/kaggle/input/hehehe/after_urban.jpg"  # Example path
    city_name = "delhi"  # Example city

    # Fetch pre-urbanization climate data
    pre_climate_data = get_weather_data(city_name)
    if pre_climate_data is None:
        print("Weather data could not be retrieved.")
        return

    # Calculate pre and post urbanization vegetative land percentage
    print("Calculating pre-urbanization vegetative land percentage...")
    pre_vegetative_percentage = calculate_vegetative_land(pre_urbanization_image_path)
    print(f"Pre-urbanization vegetative land percentage: {pre_vegetative_percentage}%")
    
    print("Calculating post-urbanization vegetative land percentage...")
    post_vegetative_percentage = calculate_vegetative_land(post_urbanization_image_path)
    print(f"Post-urbanization vegetative land percentage: {post_vegetative_percentage}%")

    # Calculate LVR
    lvr = calculate_lvr(pre_vegetative_percentage, post_vegetative_percentage)
    print(f"Land Vegetation Ratio (LVR): {lvr}")

    # Predict post-urbanization climate data
    post_climate_data = predict_post_urbanization_climate(pre_climate_data, lvr)

    # Calculate climate differences (delta)
    delta_temp, delta_humidity, delta_rain, delta_wind = calculate_climate_difference(pre_climate_data, post_climate_data)

    # Calculate Biodiversity Risk Score (BRS)
    weights = {'temp': 0.3, 'humidity': 0.3, 'rain': 0.2, 'wind': 0.2}
    brs = calculate_brs(delta_temp, delta_humidity, delta_rain, delta_wind, weights)

    # Prepare metrics for the LLM prompt
    metrics = {
        'biodiversity_risk_score': brs,
        'temperature_increase': delta_temp,
        'humidity_change': delta_humidity,
        'rain_change': delta_rain,
        'wind_speed_change': delta_wind,
        'pre_vegetative_percentage': pre_vegetative_percentage,
        'post_vegetative_percentage': post_vegetative_percentage
    }

    # Assume map_diff is a placeholder for comparing pre- and post-urbanization maps
    map_diff = "Observed significant reduction in green spaces in the northwestern regions."

    # Generate suggestions with the LLM
    print("Generating suggestions for optimized urbanization...")
    suggestions = generate_llm_suggestions(metrics, map_diff)
    print("Generated Suggestions for Optimized Urbanization:\n")
    print(suggestions)

# Run the main function
if __name__ == "__main__":
    main()
